# 03 - Feedforward Neural Network

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

Hyper Parameters

In [2]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

MNIST Dataset

In [6]:
train_dataset = dsets.MNIST(root='../data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data',
                           train=False,
                           transform=transforms.ToTensor())

Data Loader (input pipeline)

In [20]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Neural Network Model (1 hidden layer)

In [24]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)
?nn.Module

Loss and Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

Train the Model

In [11]:
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/600], Loss: 0.4013
Epoch [1/5], Step [200/600], Loss: 0.2914
Epoch [1/5], Step [300/600], Loss: 0.1442
Epoch [1/5], Step [400/600], Loss: 0.2239
Epoch [1/5], Step [500/600], Loss: 0.2241
Epoch [1/5], Step [600/600], Loss: 0.2043
Epoch [2/5], Step [100/600], Loss: 0.1885
Epoch [2/5], Step [200/600], Loss: 0.2191
Epoch [2/5], Step [300/600], Loss: 0.1301
Epoch [2/5], Step [400/600], Loss: 0.1150
Epoch [2/5], Step [500/600], Loss: 0.0706
Epoch [2/5], Step [600/600], Loss: 0.1466
Epoch [3/5], Step [100/600], Loss: 0.0410
Epoch [3/5], Step [200/600], Loss: 0.0346
Epoch [3/5], Step [300/600], Loss: 0.0702
Epoch [3/5], Step [400/600], Loss: 0.1153
Epoch [3/5], Step [500/600], Loss: 0.0596
Epoch [3/5], Step [600/600], Loss: 0.0395
Epoch [4/5], Step [100/600], Loss: 0.0109
Epoch [4/5], Step [200/600], Loss: 0.0520
Epoch [4/5], Step [300/600], Loss: 0.1368
Epoch [4/5], Step [400/600], Loss: 0.0308
Epoch [4/5], Step [500/600], Loss: 0.0329
Epoch [4/5], Step [600/600], Loss:

Test the Model

In [13]:
correct = 0
total = 0
for images,labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


Save the Model

In [18]:
torch.save(net.state_dict(), 'FeedForwardModel.pkl')